In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
total_septext_list = []
total_bio_list = []
t = []
# with open('../datasets/sample_datasets/train_500.txt', encoding='UTF-8') as f:
with open('./train.txt', encoding='UTF-8') as f:
    line_lists = f.readlines()
    septext_list = []
    bio_list = []
    for line in tqdm(line_lists):

        l = line.split(' ')

        # 判断是否读完一条数据
        if len(l) == 1:
            total_septext_list.append(''.join(septext_list))
            total_bio_list.append(' '.join(bio_list))
            septext_list = []
            bio_list = []
        # 防止遇到这种情况：'  O'
        elif len(l) == 3:
            septext_list.append(' ')
            bio_list.append(l[2].rstrip('\n'))
        elif len(l) == 2:
            septext_list.append(l[0])
            bio_list.append(l[1].rstrip('\n'))
        else:
            print('异常数据')

100%|██████████| 2288790/2288790 [00:01<00:00, 1417221.44it/s]


In [3]:
len(total_septext_list),len(total_bio_list)


(40000, 40000)

In [4]:
df = pd.DataFrame({'text': total_septext_list,
                   'bio_label': total_bio_list})

In [5]:
from sklearn.model_selection import train_test_split
train_data_df, dev_data_df = train_test_split(df, test_size = 0.1, random_state=1998)

In [6]:
train_data_df.index = list(range(len(train_data_df)))

In [7]:
train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36000 entries, 0 to 35999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       36000 non-null  object
 1   bio_label  36000 non-null  object
dtypes: object(2)
memory usage: 843.8+ KB


In [8]:
len(list(train_data_df['text'][110])),len(train_data_df['bio_label'][110].split(' '))

(62, 62)

In [9]:
with open('train_9.txt', 'w', encoding='utf-8') as f:
    for i in tqdm(range(len(train_data_df))):
        t = list(train_data_df['text'][i])
        b = train_data_df['bio_label'][i].split(' ')
        assert len(t)==len(b)
        for j in range(len(t)):
            f.write(t[j])
            f.write(' ')
            f.write(b[j])
            f.write('\n')
        f.write('\n')

100%|██████████| 36000/36000 [00:01<00:00, 19108.82it/s]


In [10]:
train_data_df.to_excel('./df1.xlsx',index=False)

In [11]:
import os
from ark_nlp.factory.utils.conlleval import get_entity_bio
datalist = []
with open('../datasets/preliminary_contest_datasets/train_data/train.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    lines.append('\n')

    text = []
    labels = []
    label_set = set()

    for line in lines:
        if line == '\n':
            text = ''.join(text)
            entity_labels = []
            for _type, _start_idx, _end_idx in get_entity_bio(labels, id2label=None):
                entity_labels.append({
                    'start_idx': _start_idx,
                    'end_idx': _end_idx,
                    'type': _type,
                    'entity': text[_start_idx: _end_idx + 1]
                })

            if text == '':
                continue

            datalist.append({
                'text': text,
                'label': entity_labels,
                'BIO': labels
            })

            text = []
            labels = []

        elif line == '  O\n':
            text.append(' ')
            labels.append('O')
        else:
            line = line.strip('\n').split()
            if len(line) == 1:
                term = ' '
                label = line[0]
            else:
                term, label = line
            text.append(term)
            label_set.add(label.split('-')[-1])
            labels.append(label)
train_data_df, dev_data_df = train_test_split(pd.DataFrame(datalist), test_size = 0.1, random_state=1998)
train_data_df.to_excel('./df2.xlsx', index=False)